## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [1]:
# requests 라이브러리 설치여부 확인
!pip show requests

Name: requests
Version: 2.32.3
Summary: Python HTTP for Humans.
Home-page: https://requests.readthedocs.io
Author: Kenneth Reitz
Author-email: me@kennethreitz.org
License: Apache-2.0
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: certifi, charset-normalizer, idna, urllib3
Required-by: anaconda-catalogs, anaconda-client, anaconda-cloud-auth, anaconda-project, conda, conda-build, conda-repo-cli, conda_package_streaming, cookiecutter, datashader, jupyterlab_server, panel, requests-file, requests-toolbelt, Sphinx, streamlit, tldextract


In [2]:
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.3
Summary: Screen-scraping library
Home-page: https://www.crummy.com/software/BeautifulSoup/bs4/
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: MIT License
Location: C:\Users\user\anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [3]:
# reqeusts, bs4 import
import requests
# BeautifulSoup 클래스 import
import bs4
# BeautifulSoup 클래스 import
from bs4 import BeautifulSoup

In [4]:
# requests, bs4 버전 확인하기
print(f'requests 버전 = {requests.__version__}')
print(f'bs4 버전 = {bs4.__version__}')

requests 버전 = 2.32.3
bs4 버전 = 4.12.3


### 1. 뉴스 제목 추출하기

In [7]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}
sid = 104
#url = f'https://news.naver.com/section/{sid}' 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)

print(url)

https://news.naver.com/section/105


In [25]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

# requests 의 get() 함수 호출하기 
res=requests.get(url,headers=req_header)
print(res.status_code, res.ok)
print(type(res))
print(res)

# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
if res.ok:
    html = res.text
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, "html.parser")   
    #print(soup)
    # CSS 선택자
    # print(soup.select("div.sa_text a[href*='mnews/article']"))
    '''
       <div class='sa_text'> 
          <a href="https://n.news.naver.com/mnews/article/366/0001094461" class="sa_text_title">
    '''
    a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
    print(len(a_tags))
    print(type(a_tags)) #[Tag,Tag,Tag]
    # <a> 태그 리스트 순회하기
    for idx,a_tag in enumerate(a_tags,1):        
        if a_tag.text:
            title = a_tag.text.strip() # 제목
            link = a_tag['href'] # 링크
            #print(len(title))
            print(f'{idx} {title} {link}')
else:
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')

https://news.naver.com/section/105
200 True
<class 'requests.models.Response'>
<Response [200]>
44
<class 'bs4.element.ResultSet'>
1 네이버, 하이퍼클로바X 상업용 무료 공개…"생태계 확장" https://n.news.naver.com/mnews/article/079/0004047692
2 폴더블폰 역대 최다 판매 ‘신기록’…갤럭시 Z 폴드7 날았다 https://n.news.naver.com/mnews/article/016/0002502956
3 LG유플러스, 자사주 1천억원 소각·800억원 매입 https://n.news.naver.com/mnews/article/215/0001217058
4 상반기 휩쓴 휴머노이드 ETF, 어디가 잘하고 있나 https://n.news.naver.com/mnews/article/008/0005224663
5 라이너, BC카드와 협력…첫 결과물은 'AI 맛집 검색' https://n.news.naver.com/mnews/article/030/0003333800
6 과기정통부-행안부, 대전서 AI 기반 안전관리 시스템 검증 https://n.news.naver.com/mnews/article/001/0015522362
7 숲, LCK 시즌 재개 맞아 다양한 이벤트 진행 https://n.news.naver.com/mnews/article/029/0002969976
8 대기업도 개인정보 유출 '참사'..실시간 유출 알림으로 보안 의식 업 https://n.news.naver.com/mnews/article/018/0006071030
9 최휘영 문체부장관 후보자 "게임이용장애 질병코드 등재 신중해야" https://n.news.naver.com/mnews/article/031/0000950817
10 AI 발전에 ‘진심’ 네이버…‘하이퍼클로바X’ 추론 모델 상업용 무료 공개 https://n.news.naver.com/mnew

### 1.1 뉴스제목 추출하는 함수 선언하기

In [27]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section):  #print_new(103,'생활/문화') 
    sid = section_dict[section]
    url = f'https://news.naver.com/section/{sid}'
    print(f'{url} {section} 뉴스')

    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    # requests 의 get() 함수 호출하기 
    res=requests.get(url,headers=req_header)
    print(res.status_code, res.ok)
    if res.ok:
        html = res.text
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
        print(len(a_tags))
        # <a> 태그 리스트 순회하기
        for idx,a_tag in enumerate(a_tags,1):        
            if a_tag.text:
                title = a_tag.text.strip() # 제목
                link = a_tag['href'] # 링크
                #print(len(title))
                print(f'{idx} {title} {link}')
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')       
    

In [29]:
print_news('세계')

https://news.naver.com/section/104 세계 뉴스
200 True
46
1 '관세부과 D-11' 위성락 실장, 막바지 물밑 협상...미 재무 '합의의 질' 강조 https://n.news.naver.com/mnews/article/052/0002222589
2 주4일제 했더니 퇴사 생각이 사라졌다 https://n.news.naver.com/mnews/article/014/0005380313
3 방글라 공군 훈련기, 수업 중인 학교로 추락... 최소 25명 사망 https://n.news.naver.com/mnews/article/030/0003333868
4 벼랑끝 또 만난 원수… 아소, 이시바 퇴진 압박 https://n.news.naver.com/mnews/article/021/0002724372
5 한국, 국제수학올림피아드서 종합 3위 https://n.news.naver.com/mnews/article/366/0001094447
6 美상무부 중국계 직원, 가족 만나러 中 갔다가 출금 https://n.news.naver.com/mnews/article/020/0003649399
7 러시아·우크라이나, 튀르키예서 7주 만에 평화협상 재개 https://n.news.naver.com/mnews/article/032/0003384420
8 파월 쫒겨나면 美 국채 0.5%p 폭락 가능...해임 가능성 여전 https://n.news.naver.com/mnews/article/014/0005380629
9 트럼프의 뒤끝…"전용기에 WSJ 타지마"[글로벌 왓] https://n.news.naver.com/mnews/article/011/0004511892
10 “日 여행간 딸, 입원했대서 돈 보냈는데”…1개월째 행방불명 https://n.news.naver.com/mnews/article/009/0005528814
11 中 최신 스텔스 J-35, 실전 배치 임박 사진 확산 https://n.news.naver.com/mnews/articl

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [37]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=751168&no=217&week=fri',
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/751168/217/20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/751168/217/20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/751168/217/20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_3.jpg'
]

for img_url in img_urls:
    
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url,headers=req_header)
    print(res.ok, res.status_code)
    if res.ok:
        # binary 응답 데이터 가져오기
        img_data =res.content
        # url에서 파일명만 추출하기
        file_name =os.path.basename(img_url)
        print(file_name)
        # binday data를 file에 write하기
        with open(file_name, 'wb') as file:
            print(f'writing to {file_name}({len(img_data):,}bytes)')
            file.write(img_data)


True 200
20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_1.jpg
writing to 20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_1.jpg(179,870bytes)
True 200
20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_2.jpg
writing to 20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_2.jpg(99,573bytes)
True 200
20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_3.jpg
writing to 20250626115110_5a5df90bf1ded29dd89a2237b9b85aac_IMAG01_3.jpg(162,241bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [40]:
import requests
from bs4 import BeautifulSoup
webtoon_url ='https://comic.naver.com/webtoon/detail?titleId=769209&no=149&week=finish  '
req_header={ 
    'referer':webtoon_url,
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}
res = requests.get(webtoon_url)
print(res.status_code)
if res.ok: 
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("img[src$='.jpg']")))
    #https://image-comic.pstatic.net/webtoon
    print(len(soup.select("img[src^='https://image-comic.pstatic.net/webtoon']")))
    print(len(soup.select("img[src*='IMAG01']"))) # 일치함

200
232
232
74


### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [ ]:
import requests

upload_files = {
    
}
#print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.



200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


In [ ]:
#  사용자 입력값 검증 요청



* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


### 5. 블로그 검색하기

In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
